In [1]:
from tlspt.datamodules.components.numpy_dataset import NumpyDataset
from tlspt.datamodules.components.base_site import BaseSiteDataset
from tlspt.datamodules.components.octree_dataset import OctreeDataset
from tlspt.datamodules.components.merged_dataset import MergedOctreeDataset
from tlspt.transforms import TLSSampler, UniformDownsample
from tlspt.datamodules.single_datamodule import SingleDataModule

from torch.utils.data import DataLoader


from matplotlib import pyplot as plt

from loguru import logger

import sys
import numpy as np
import time

logger.remove()  # Remove the default handler
logger.add(sys.stdout, level="INFO")

1

In [2]:
site_dataset = OctreeDataset(split_file='../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                 split='train',
                                 feature_names=['red', 'green', 'blue'],
                                 features_to_normalize=['red', 'green', 'blue'],
                                 scale=1.5,
                                 transform=UniformDownsample(num_points=1024, replace="resample_as_req"),
                                 plots_keep=['SPA01'])

2024-11-22 19:46:04.687 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from ../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-22 19:46:04.690 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 14 folders in ../data/plot_octrees/allen-spain/octrees/
2024-11-22 19:46:04.691 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 14 plots for 'train' out of 14 plots defined in split file
2024-11-22 19:46:04.691 | INFO     | tlspt.structures.file_octree:__init__:155 - Initializing octree from <cl

In [3]:
site_dataset.octrees[0]

In [4]:
len(site_dataset.nodes)

1

In [5]:
print(f"Number of nodes in DS plot 0: {len(site_dataset.nodes[0])}")

Number of nodes in DS plot 0: 1492


In [6]:
len(site_dataset.leaf_nodes[0])  #e.g. site_dataset.leaf_nodes[number of plot][number of parent node]
num_leaves_per_node = [len(x) for x in site_dataset.leaf_nodes[0]]
print(f"Number of leaves total in DS plot 0: {sum(num_leaves_per_node)}")

Number of leaves total in DS plot 0: 1492


In [7]:
len(site_dataset)

1492

In [8]:
site_dataset.load_item(0)

In [9]:
site_dataset.prepare_data(force_compute=False)

2024-11-22 19:46:05.885 | INFO     | tlspt.utils:prepare_data:95 - reading stats from ../data/plot_octrees/allen-spain/octrees/stats/stats_128a15cc4e5f0.pkl
2024-11-22 19:46:05.886 | INFO     | tlspt.utils:prepare_data:96 - for dataset OctreeDataset.{'features_to_normalize': ['red', 'green', 'blue']}.train.128a15cc4e5f0
2024-11-22 19:46:05.888 | INFO     | tlspt.utils:prepare_data:105 - mean: tensor([0.5721, 0.5507, 0.5738])
2024-11-22 19:46:05.889 | INFO     | tlspt.utils:prepare_data:106 - std: tensor([0.3137, 0.3621, 0.2952])
2024-11-22 19:46:05.889 | INFO     | tlspt.utils:prepare_data:110 - torch.float32
2024-11-22 19:46:05.890 | INFO     | tlspt.utils:prepare_data:114 - Dataset has 3 features named ['red', 'green', 'blue']. 
 Normalizing ['red', 'green', 'blue'] by mean+std. 
2024-11-22 19:46:05.891 | INFO     | tlspt.utils:prepare_data:117 - 3 channels will be zero centered and scaled to [-1,1].


In [10]:
site_dataset[9]['features'].shape

torch.Size([1024, 3])

In [11]:
site_dataset[9]['points'].shape

torch.Size([1024, 3])

In [12]:
site_dataset[0]

{'points': tensor([[ 0.3254,  0.3766,  0.1012],
         [-0.1289,  0.3342,  0.1315],
         [-0.1442, -0.4888, -0.0962],
         ...,
         [-0.3268, -0.7928, -0.1168],
         [-0.1392, -0.5985, -0.1441],
         [-0.0803,  0.4192,  0.1624]]),
 'features': tensor([[-0.8238,  0.7533, -0.7215],
         [-0.8238,  0.7533, -0.7215],
         [-0.6238,  1.2082, -0.4824],
         ...,
         [-0.6238,  1.2082, -0.4824],
         [-0.6238,  1.2082, -0.4824],
         [-0.8238,  0.7533, -0.7215]]),
 'lengths': 1024}

In [13]:
length = site_dataset[9]['lengths']
mat = np.array(site_dataset[9]['points'])  # Ensure 'mat' is a NumPy array

print(mat[0:5])  # Print first 5 points
print()
print(mat[length:length+5])  # Print points from 'length' to 'length + 5'

# Find indices of rows in 'mat' that are equal to the first row
indices = np.where(np.all(mat == mat[0], axis=1))[0]
print("Indices of rows equal to the first row:", indices)

print(len(mat))

[[-0.1063687  -0.00955277  0.9593539 ]
 [-0.19007088  0.05334659  0.9625488 ]
 [-0.06148773  0.03218045 -0.06620518]
 [ 0.2579567  -0.04628844 -0.5633653 ]
 [-0.13718328  0.05365443  1.        ]]

[[-0.1063687  -0.00955277  0.9593539 ]
 [-0.19007088  0.05334659  0.9625488 ]
 [-0.06148773  0.03218045 -0.06620518]
 [ 0.2579567  -0.04628844 -0.5633653 ]
 [-0.13718328  0.05365443  1.        ]]
Indices of rows equal to the first row: [  0 545]
1024


In [14]:
dataloader = DataLoader(site_dataset, batch_size=128, shuffle=True)

In [15]:
batch = next(iter(dataloader))

In [16]:
batch['lengths']

tensor([1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,  673, 1024, 1024,  566,
        1024, 1024, 1024,  870, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,  950, 1024, 1024,
         515, 1024, 1024, 1024, 1024, 1024, 1024, 1024])

In [17]:
merged_dataset = MergedOctreeDataset(
    split_files=['/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-poland/octrees/allen-poland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv'],
    split='train',
    scales=[1.5, 1.5, 1.5],
    feature_names=None,
    normalize=True,
    transform=UniformDownsample(num_points=1024, replace="resample_as_req")
)

2024-11-22 19:46:10.238 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-22 19:46:10.240 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 12 folders in /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/
2024-11-22 19:46:10.241 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 12 plots for 'train' out of 12 plots defined in split f

In [18]:
merged_dataloader = DataLoader(merged_dataset, batch_size=128, shuffle=True, num_workers=0)

In [19]:
%%time 
for i, batch in enumerate(merged_dataloader):
    if i > 5:
        break
    pass

CPU times: user 6min 21s, sys: 13.8 s, total: 6min 35s
Wall time: 26.2 s


In [20]:
from tlspt.transforms import HDF5ChunkSampler

In [21]:
sampler = HDF5ChunkSampler(8192)

In [22]:
idx = sampler(1024, 53111)
for x in idx:
    print(x, x[1]-x[0])

print(sum([x[1]-x[0] for x in idx]))

(12288, 13312) 1024
(14336, 15360) 1024
(26624, 27648) 1024
(30720, 31744) 1024
(33792, 34816) 1024
(35840, 36864) 1024
(38912, 39936) 1024
(43008, 44032) 1024
8192


In [23]:
from tlspt.datamodules.components.octree_dataset import OctreeDatasetHdf5
from tlspt.transforms import Padder

import importlib

In [24]:
site_dataset_hdf5 = OctreeDatasetHdf5(split_file='../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                 split='train',
                                 feature_names=['red', 'green', 'blue', 'scalar_pathlength', 'scalar_distance', 'scalar_label'],
                                 features_to_normalize=['red', 'green', 'blue'],
                                 scale=1.5,
                                 idx_sampler=HDF5ChunkSampler(8192),
                                 transform=Padder(num_points=8192),
                                 plots_keep=['SPA01'])

2024-11-22 19:46:40.910 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from ../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-22 19:46:40.912 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 14 folders in ../data/plot_octrees/allen-spain/octrees/
2024-11-22 19:46:40.913 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 14 plots for 'train' out of 14 plots defined in split file
2024-11-22 19:46:40.914 | INFO     | tlspt.structures.file_octree:__init__:155 - Initializing oct

In [25]:
p, f = site_dataset_hdf5.load_item(7)
print(p.shape, f.shape)

torch.Size([8192, 3]) torch.Size([8192, 6])


In [26]:
site_dataset_hdf5.prepare_data(force_compute=False)

2024-11-22 19:46:41.214 | INFO     | tlspt.utils:prepare_data:246 - reading stats from ../data/plot_octrees/allen-spain/octrees/stats/stats_333b10a0a3137.pkl
2024-11-22 19:46:41.215 | INFO     | tlspt.utils:prepare_data:247 - for dataset OctreeDatasetHdf5.{'features_to_normalize': ['red', 'green', 'blue']}.train.333b10a0a3137
2024-11-22 19:46:41.216 | INFO     | tlspt.utils:prepare_data:256 - mean: tensor([0.5659, 0.5654, 0.5750,    nan,    nan,    nan])
2024-11-22 19:46:41.217 | INFO     | tlspt.utils:prepare_data:257 - std: tensor([0.3076, 0.3576, 0.2889,    nan,    nan,    nan])
2024-11-22 19:46:41.218 | INFO     | tlspt.utils:prepare_data:261 - torch.float32
2024-11-22 19:46:41.218 | INFO     | tlspt.utils:prepare_data:265 - Dataset has 6 features named ['red', 'green', 'blue', 'scalar_pathlength', 'scalar_distance', 'scalar_label']. 
 Normalizing ['red', 'green', 'blue'] by mean+std. 
2024-11-22 19:46:41.218 | INFO     | tlspt.utils:prepare_data:268 - 3 channels will be zero cente

In [27]:
print(site_dataset_hdf5[1]['points'].shape)
print(site_dataset_hdf5[1]['features'].shape)
print(site_dataset_hdf5[1]['lengths'])

torch.Size([8192, 3])
torch.Size([8192, 6])
3864


In [28]:
# #3D scatter plot 
# fig = plt.figure(figsize=(10, 10))

# ax = fig.add_subplot(111, projection='3d')
# i=np.random.randint(0, len(site_dataset_hdf5))
# ax.scatter(site_dataset_hdf5[i]['points'][:, 0], site_dataset_hdf5[i]['points'][:, 1], site_dataset_hdf5[i]['points'][:, 2], c=site_dataset_hdf5[i]['features'][:, -1], cmap='viridis', alpha=1, s=5)

In [29]:
merged_dataset_hdf5 = MergedOctreeDataset(
    split_files=['/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-poland/octrees/allen-poland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv'],
    split='train',
    scales=[1.5, 1.5, 1.5],
    feature_names=None,
    normalize=True,
    hdf5=True,
    idx_sampler=HDF5ChunkSampler(8192),
    transform=Padder(num_points=8192)
)

2024-11-22 19:46:41.723 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDatasetHdf5(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-22 19:46:41.725 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDatasetHdf5(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 12 folders in /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/
2024-11-22 19:46:41.726 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDatasetHdf5(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 12 plots for 'train' out of 12 plots define

In [30]:
merged_dataloader_hdf5 = DataLoader(merged_dataset_hdf5, batch_size=128, shuffle=True, num_workers=0)

In [33]:
%%time 
for i, batch in enumerate(merged_dataloader):
    if i > 10:
        break
    pass

CPU times: user 11min 8s, sys: 23.1 s, total: 11min 31s
Wall time: 46.1 s


In [34]:
%%time 
for i, batch in enumerate(merged_dataloader_hdf5):
    if i > 10:
        break
    pass

CPU times: user 4min 55s, sys: 10.6 s, total: 5min 6s
Wall time: 41.1 s
